In [ ]:
from pyetp.client import connect
from pyetp.config import SETTINGS
from pyetp.types import *
import xtgeo

SETTINGS.dataspace = 'demo/pss-data-gateway'
SETTINGS.etp_url = 'ws://localhost:9100'
SETTINGS.application_name ='etpTest'
SETTINGS.application_version='0.0.1'

token = ""
async with connect(timeout=60) as client:
        await client.put_dataspaces_no_raise(client.default_dataspace_uri) # create new dataspace if not exist
        surf = xtgeo.surface_from_file('../data/test.gri', fformat='irap_binary')
        epsg_code = 23031
        urls = await client.put_xtgeo_surface(surf, epsg_code, client.default_dataspace_uri) # chunked upload using subarray if size too large


In [ ]:
for i in urls:
    print(i.raw_uri)


In [ ]:
async with connect(authorization=token, timeout=60) as client:
    gri, = await client.get_resqml_objects(urls[2]) # parallelized using subarray
    arr = await client.get_array(DataArrayIdentifier(
                uri=str(urls[0]), pathInResource=gri.grid2d_patch.geometry.points.zvalues.values.path_in_hdf_file
            )) # parallelized using subarray

In [ ]:
import random
# Get map value from point
x_max = surf.xori + (surf.xinc*surf.ncol)
y_max = surf.yori + (surf.yinc*surf.nrow)
x = random.uniform(surf.xori, x_max) 
y= random.uniform(surf.yori, y_max) 
async with connect(authorization=token, timeout=60) as client:
    linear = await client.get_surface_value_x_y(urls[0], urls[2], x,y,"linear")
    xtgeo_linear = surf.get_value_from_xy((x,y))
print(xtgeo_linear,linear)
